# D. State stabilization in a noisy environment with Jaynes-Cummings controls

Here the result is as follows: The algorithm optimizes the params, such that the POVM always outputs 1, implying that the measurement leaves the target state invariant. this is what we are indeed seeing when printing the measurement outcome and its probability, when batching however, the optimizer struggles to converge.

Also, lookup here is much better than nn for the same hyperparameters.

This is actually a perfect example, of feedback grape modifying the params so that a certain measurement sequence will always be output because this measurement sequence is the one that is going to lead to the best fidelity

In [1]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [ ]:
from feedback_grape.fgrape import optimize_pulse
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

In [3]:
N_cav = 20

## Here, dividing alpha into real and imaginary parts complicates the optimization and converges at 0.89 while if we do not use the imaginary part it converges at 0.999

In [4]:
def qubit_unitary(alphas):
    alpha_re, alpha_im = alphas
    alpha = alpha_re + 1j * alpha_im
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [5]:
def qubit_cavity_unitary(betas):
    beta_re, beta_im = betas
    beta = beta_re + 1j * beta_im
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

### povm_measure_operator (callable): <br>
    - It should take a measurement outcome and list of params as input
    - The measurement outcome options are either 1 or -1

In [6]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma, delta = params
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2 * identity(2*N_cav)
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

In [7]:
from feedback_grape.utils.states import coherent

alpha = 3
psi_target = tensor(
    coherent(N_cav, alpha)
    + coherent(N_cav, -alpha)
    + coherent(N_cav, 1j * alpha)
    + coherent(N_cav, -1j * alpha),
    basis(2),
)  # 4-legged state

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)
rho_target = psi_target @ psi_target.conj().T

In [8]:
rho_target.shape

(40, 40)

### It is important to test what the POVM probability is, to check if your state is normalized. if the probability is bounded between 0 and 1 then normalized
This completeness condition on the measurement operators is automatically checked for the `initial_params` in `optimize_pulse`.

In [9]:
# Answer: should one normalize within the optimization just in case?
# --> no so that user is not misled into thinking that his unnormalized state
# is working properly
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho_target,
    +1,
    povm_measure_operator(+1, params=[0.058, jnp.pi / 2]),
    povm_measure_operator(-1, params=[0.058, jnp.pi / 2]),
    evo_type="density",
)

Array(0.09357621, dtype=float64)

In [10]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho_target, C_target=rho_target, evo_type="density"))

1.0


### Without dissipation

In [ ]:
# Here the loss directly corressponds to the -fidelity (when converging) because log(1) is 0 and
# the algorithm is choosing params that makes the POVM generate prob = 1
from feedback_grape.fgrape import Gate, Decay
import jax

key1, key2, key3 = jax.random.split(jax.random.PRNGKey(42), 3)
measure = Gate(
    gate=povm_measure_operator,
    initial_params=jax.random.uniform(
        key1,
        shape=(2,),  # 2 for gamma and delta
        minval=-2 * jnp.pi,
        maxval=2 * jnp.pi,
    ),
    measurement_flag=True,
    # param_constraints=[[0, jnp.pi], [-2*jnp.pi, 2*jnp.pi]],
)

qub_unitary = Gate(
    gate=qubit_unitary,
    initial_params=jax.random.uniform(
        key2,
        shape=(2,),  # 2 for real(alpha) and imag(alpha)
        minval=-2 * jnp.pi,
        maxval=2 * jnp.pi,
    ),
    measurement_flag=False,
    # param_constraints=[[-2*jnp.pi, 2*jnp.pi], [-2*jnp.pi, 2*jnp.pi]],
)

qub_cav = Gate(
    gate=qubit_cavity_unitary,
    initial_params=jax.random.uniform(
        key3,
        shape=(2,),  # 2 for real(beta) and imag(beta)
        minval=-2 * jnp.pi,
        maxval=2 * jnp.pi,
    ),
    measurement_flag=False,
    # param_constraints=[[-jnp.pi, jnp.pi], [-jnp.pi, jnp.pi]],
)

system_params = [measure, qub_unitary, qub_cav]

for reward_weights in [(1,1), (0,1)]:
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-6,
        learning_rate=0.001,
        evo_type="density",
        batch_size=1,
        eval_time_steps=5,
    )

    print(f"reward_weights: {reward_weights}\n fidelity@t=2: {result.fidelity_each_timestep[2]}\n mean fidelity each timestep: {jnp.mean(jnp.array(result.fidelity_each_timestep), axis=1)}\n")

reward_weights: (1, 1)
 fidelity@t=2: [0.61147643 0.61147643 0.61147643 0.61147643 0.61147643 0.61147643
 0.61147643 0.61147643 0.61147643 0.61147643]
 mean fidelity each timestep: [1.         0.05440354 0.61147643 0.02560076 0.23348661 0.14675473]

reward_weights: (0, 1)
 fidelity@t=2: [0.70072227 0.70072227 0.70072227 0.70072227 0.70072227 0.70072227
 0.70072227 0.70072227 0.70072227 0.70072227]
 mean fidelity each timestep: [1.         0.10651565 0.70072227 0.08212367 0.15468819 0.26903952]



In [12]:
result.optimized_trainable_parameters["initial_params"]

[Array([-3.25968973, -1.22546366], dtype=float64),
 Array([3.04255722, 0.59257698], dtype=float64),
 Array([-1.58503058,  2.06652483], dtype=float64)]

In [13]:
result.returned_params

[[Array([[-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366],
         [-3.25968973, -1.22546366]], dtype=float64),
  Array([[3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519],
         [3.74446264, 1.05778519]], dtype=float64),
  Array([[-1.18880838,  1.68717978],
         [-1.18880838,  1.68717978],
         [-1.18880838,  1.68717978],
         [-1.18880838,  1.68717978],
         [-1.18880838,  1.68717978],
         [-1.18880838,  1.68717978],
         [-1.18880838,  

In [14]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0
fidelity of state 0: 0.008146989341838724
fidelity of state 1: 0.4296580047885219
fidelity of state 2: 0.4296580047885219
fidelity of state 3: 0.019468980947527772
fidelity of state 4: 0.4296580047885219
fidelity of state 5: 0.07668419535038538
fidelity of state 6: 0.4296580047885219
fidelity of state 7: 0.4296580047885219
fidelity of state 8: 0.4296580047885219
fidelity of state 9: 0.008146989341838724


### With Dissipation

In [15]:
# Note if tsave = jnp.linspace(0, 1, 1) = [0.0] then the decay is not applied ?
# because the first time step has the original non decayed state
# Question: Here I should not use any sort of Hamiltonian or tspan or something?

decay = Decay(
    c_ops=[tensor(identity(N_cav), jnp.sqrt(0.1) * sigmam())],
)
system_params = [decay, measure, qub_unitary, qub_cav]

for reward_weights in [(1,1), (0,1)]:
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-6,
        learning_rate=0.001,
        evo_type="density",
        batch_size=1,
        eval_time_steps=5,
    )

    print(f"reward_weights: {reward_weights}\n fidelity@t=2: {result.fidelity_each_timestep[2]}\n mean fidelity each timestep: {jnp.mean(jnp.array(result.fidelity_each_timestep), axis=1)}\n")

reward_weights: (1, 1)
 fidelity@t=2: [0.55096597 0.55096597 0.55096597 0.05109454 0.55096597 0.55096597
 0.55096597 0.55096597 0.55096597 0.55096597]
 mean fidelity each timestep: [1.         0.09291582 0.50097883 0.06732321 0.18326516 0.08771339]

reward_weights: (0, 1)
 fidelity@t=2: [0.60162457 0.60162457 0.60162457 0.60162457 0.60162457 0.60162457
 0.60162457 0.60162457 0.60162457 0.60162457]
 mean fidelity each timestep: [1.         0.10592718 0.60162457 0.08587846 0.16084106 0.11607979]



In [16]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0
fidelity of state 0: 0.15125949286446827
fidelity of state 1: 0.013853869884188937
fidelity of state 2: 0.013853869884188937
fidelity of state 3: 0.24132876335064737
fidelity of state 4: 0.24132876335064737
fidelity of state 5: 0.013853869884188937
fidelity of state 6: 0.24132876335064737
fidelity of state 7: 0.03411857686442019
fidelity of state 8: 0.09974453405941826
fidelity of state 9: 0.11012737621612621


In [17]:
result.returned_params

[[Array([[-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491],
         [-3.22390513, -1.25310491]], dtype=float64),
  Array([[3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396],
         [3.46904782, 1.00731396]], dtype=float64),
  Array([[-1.23277087,  1.70909133],
         [-1.23277087,  1.70909133],
         [-1.23277087,  1.70909133],
         [-1.23277087,  1.70909133],
         [-1.23277087,  1.70909133],
         [-1.23277087,  1.70909133],
         [-1.23277087,  

In [18]:
result.optimized_trainable_parameters["initial_params"]

[Array([-3.22390513, -1.25310491], dtype=float64),
 Array([3.04255722, 0.59257698], dtype=float64),
 Array([-1.58503058,  2.06652483], dtype=float64)]